<a href="https://colab.research.google.com/github/YWchris/Effect-of-heterogeneity-and-cluster-analysis/blob/main/Random_Sample_Chest_X_ray_ROP_Christine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Random sample the same number of images from the training set as the number of images in each cluster
- Train binary CNN for each sample (Prevent the confounding effect of sample size)

In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow==2.9.0

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: flatbuffers
    Found existing installation

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv('gdrive/My Drive/Chest X-Ray NIH/sample_labels.csv')
len(df)

5606

In [ ]:
df.head(10)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,No Finding,1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168
5,00000040_003.png,Consolidation|Mass,3,40,068Y,M,PA,2500,2048,0.168,0.168
6,00000042_002.png,No Finding,2,42,071Y,M,AP,3056,2544,0.139,0.139
7,00000057_001.png,No Finding,1,57,071Y,M,AP,3056,2544,0.139,0.139
8,00000061_002.png,Effusion,2,61,077Y,M,PA,2992,2991,0.143,0.143
9,00000061_019.png,No Finding,19,61,077Y,M,AP,3056,2544,0.139,0.139


In [ ]:
data2 = df[df['Finding Labels'].str.contains('Cardiomegaly')]
print(len(data2))
data3 = df.loc[(df['Finding Labels'] == 'No Finding')]
data3 = data3.iloc[:141]
print(len(data3))
#frame_data = [data2, data3]
#data_w = pd.concat(frame_data)
#data_w = data_w.sample(frac = 1, random_state=42)
#len(data_w)
#data_w.head()
data2.head()

141
141


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168
19,00000116_013.png,Cardiomegaly|Consolidation,13,116,075Y,F,AP,2500,2048,0.168,0.168
21,00000116_032.png,Cardiomegaly|Consolidation,32,116,075Y,F,AP,2500,2048,0.168,0.168
22,00000116_040.png,Cardiomegaly|Consolidation,40,116,075Y,F,AP,2500,2048,0.168,0.168


In [ ]:
data_1 = data3.loc[(data3['Finding Labels'] == 'No Finding') & (data3['Patient Gender'] == 'M')]
data_2 = data3.loc[(data3['Finding Labels'] == 'No Finding') & (data3['Patient Gender'] == 'F')]
#data_1 = data_1.iloc[:422]
#data_2 = data_2.iloc[:422]
print(len(data_1), len(data_2))

data_3 = data2.loc[(data2['Finding Labels'].str.contains('Cardiomegaly')) & (data2['Patient Gender'] == 'M')]
data_4 = data2.loc[(data2['Finding Labels'].str.contains('Cardiomegaly')) & (data2['Patient Gender'] == 'F')]
#data_3 = data_3.iloc[:422]
print(len(data_3), len(data_4))
frame_data = [data_1, data_2, data_3, data_4]
dataT = pd.concat(frame_data)
dataT = dataT.sample(frac = 1, random_state=42)
print(len(dataT))
dataT.head()

68 73
68 73
282


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
99,00000554_000.png,No Finding,0,554,055Y,F,PA,2726,2669,0.143,0.143
105,00000598_000.png,No Finding,0,598,078Y,F,PA,2048,2500,0.168,0.168
3190,00015693_007.png,Cardiomegaly|Edema,7,15693,052Y,M,AP,2500,2048,0.168,0.168
200,00001101_022.png,No Finding,22,1101,045Y,F,AP,3056,2544,0.139,0.139
4103,00020213_045.png,Atelectasis|Cardiomegaly|Consolidation|Effusio...,45,20213,072Y,F,AP,3056,2544,0.139,0.139


In [ ]:
dataT['Result'] = dataT['Finding Labels'].apply(lambda x: 0 if 'No Finding' in x else 1)
dataT.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,Result
99,00000554_000.png,No Finding,0,554,055Y,F,PA,2726,2669,0.143,0.143,0
105,00000598_000.png,No Finding,0,598,078Y,F,PA,2048,2500,0.168,0.168,0
3190,00015693_007.png,Cardiomegaly|Edema,7,15693,052Y,M,AP,2500,2048,0.168,0.168,1
200,00001101_022.png,No Finding,22,1101,045Y,F,AP,3056,2544,0.139,0.139,0
4103,00020213_045.png,Atelectasis|Cardiomegaly|Consolidation|Effusio...,45,20213,072Y,F,AP,3056,2544,0.139,0.139,1


In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(dataT,
                                   test_size = 0.20,
                                   random_state = 2023,
                                   stratify = dataT[['Result']])

In [ ]:
print(len(train_df), len(test_df))

225 57


In [ ]:
train_e = train_df.loc[(train_df["Result"] == 1)]["Image Index"].values
train_n = train_df.loc[(train_df["Result"] == 0)]["Image Index"].values

In [ ]:
test_e = test_df.loc[(test_df["Result"] == 1)]["Image Index"].values
test_n = test_df.loc[(test_df["Result"] == 0)]["Image Index"].values

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
REQUIRES_PYTHON = '>=3.5.0'
VERSION = '0.1.0'
from torch import optim, nn
from torchvision import models, transforms

In [ ]:
data_path='/content/drive/MyDrive/Chest X-Ray NIH/sample/images'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("On which device we are on:{}".format(device))

from google.colab import drive
drive.mount('/content/drive')

from pandas.core.common import random_state
import cv2
import glob
from PIL import Image
image_size=224
labels = []
dataset = []
random.seed(1)
def create_dataset(image_category,label):
    for img in tqdm(image_category):
        image_path = os.path.join(data_path,img)
        try:
            image = cv2.imread(image_path,cv2.IMREAD_COLOR)
            image = cv2.resize(image,(image_size,image_size))
        except:
            continue

        dataset.append([image,label])
    random.shuffle(dataset)
    return dataset

dataset = create_dataset(train_e,1)

dataset = create_dataset(train_n,0)
x = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3) #features
y = np.array([i[1] for i in dataset]) #labels
len(x)
len(y)

On which device we are on:cpu
Mounted at /content/drive


  0%|          | 0/112 [00:00<?, ?it/s]

  0%|          | 0/113 [00:00<?, ?it/s]

225

In [ ]:
data_path='/content/drive/MyDrive/Chest X-Ray NIH/sample/images'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("On which device we are on:{}".format(device))

from google.colab import drive
drive.mount('/content/drive')

from pandas.core.common import random_state
import cv2
import glob
image_size=224
labels = []
dataset = []
random.seed(1)
def create_dataset(image_category,label):
    for img in tqdm(image_category):
        image_path = os.path.join(data_path,img)
        try:
            image = cv2.imread(image_path,cv2.IMREAD_COLOR)
            image = cv2.resize(image,(image_size,image_size))
        except:
            continue

        dataset.append([np.array(image),np.array(label)])
    random.shuffle(dataset)
    return dataset

dataset = create_dataset(test_e,1)

dataset = create_dataset(test_n,0)

x_testall = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3) #features
y_testall = np.array([i[1] for i in dataset]) #labels
len(x_testall)
len(y_testall)

On which device we are on:cpu
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

57

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

datagen.fit(x)

In [ ]:
# Augmentation: Training data
batch_size = 8
num_augmented = 225

augmented_images = np.zeros((num_augmented, image_size, image_size, 3))
for i in range(num_augmented):
    img = x[i % len(x)]  # select an image from the original dataset
    augmented_images[i] = datagen.random_transform(img)

x_aug = np.concatenate((x, augmented_images), axis=0)
y_aug = np.concatenate((y, y[:num_augmented]), axis=0)
y_aug[:len(y)] = y  # repeat the original output data to match the number of samples in x_aug


In [ ]:
print(len(x_aug), len(y_aug))

450 450


In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 96
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]

    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False

    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    checkpoint = ModelCheckpoint("samplex_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=10, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=30, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])

    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/30
12/12 [==============================] - ETA: 0s - loss: 1.5384 - accuracy: 0.5625
Epoch 1: val_accuracy improved from -inf to 0.63158, saving model to samplex_efficientnet.h5
12/12 [==============================] - 14s 731ms/step - loss: 1.5384 - accuracy: 0.5625 - val_loss: 0.8538 - val_accuracy: 0.6316
Epoch 2/30
12/12 [==============================] - ETA: 0s - loss: 0.7130 - accuracy: 0.8438
Epoch 2: val_accuracy improved from 0.63158 to 0.66667, saving model to samplex_efficientnet.h5
12/12 [==============================] - 9s 740ms/step - loss: 0.7130 - accuracy: 0.8438 - val_loss: 1.2062 - val_accuracy: 0.6667
Epoch 3/30
12/12 [==============================] - ETA: 0s - loss: 0.5488 - accuracy: 0.8958
Epoch 3: val_accuracy did not improve from 0.66667
12/12 [==============================] - 7s 581ms/step - loss: 0.5488 - accuracy: 0.8958 - val_loss: 3.0187 - val_accuracy: 0.5439
Epoch 4/30
12/12 [=

In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 108
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]

    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False

    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    checkpoint = ModelCheckpoint("samplex_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=10, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=30, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])

    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


Epoch 1/30
14/14 [==============================] - ETA: 0s - loss: 1.9052 - accuracy: 0.5648
Epoch 1: val_accuracy improved from -inf to 0.56140, saving model to samplex_efficientnet.h5
14/14 [==============================] - 15s 683ms/step - loss: 1.9052 - accuracy: 0.5648 - val_loss: 1.6182 - val_accuracy: 0.5614
Epoch 2/30
14/14 [==============================] - ETA: 0s - loss: 0.4275 - accuracy: 0.8611
Epoch 2: val_accuracy improved from 0.56140 to 0.68421, saving model to samplex_efficientnet.h5
14/14 [==============================] - 9s 680ms/step - loss: 0.4275 - accuracy: 0.8611 - val_loss: 1.4645 - val_accuracy: 0.6842
Epoch 3/30
14/14 [==============================] - ETA: 0s - loss: 0.1528 - accuracy: 0.9444
Epoch 3: val_accuracy did not improve from 0.68421
14/14 [==============================] - 8s 575ms/step - loss: 0.1528 - accuracy: 0.9444 - val_loss: 1.4019 - val_accuracy: 0.6667
Epoch 4/30
14/14 [==============================] - ETA: 0s - loss: 0.0780 - accurac

2/2 [==============================] - 3s 1s/step - loss: 1.2424 - accuracy: 0.7719
Epoch 1/30
14/14 [==============================] - ETA: 0s - loss: 1.4015 - accuracy: 0.6111
Epoch 1: val_accuracy improved from -inf to 0.68421, saving model to samplex_efficientnet.h5
14/14 [==============================] - 16s 683ms/step - loss: 1.4015 - accuracy: 0.6111 - val_loss: 1.1990 - val_accuracy: 0.6842
Epoch 2/30
14/14 [==============================] - ETA: 0s - loss: 0.2958 - accuracy: 0.9167
Epoch 2: val_accuracy did not improve from 0.68421
14/14 [==============================] - 9s 663ms/step - loss: 0.2958 - accuracy: 0.9167 - val_loss: 2.0411 - val_accuracy: 0.5965
Epoch 3/30
14/14 [==============================] - ETA: 0s - loss: 0.0631 - accuracy: 0.9722
Epoch 3: val_accuracy improved from 0.68421 to 0.75439, saving model to samplex_efficientnet.h5
14/14 [==============================] - 9s 640ms/step - loss: 0.0631 - accuracy: 0.9722 - val_loss: 1.1566 - val_accuracy: 0.7544


In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 113
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]

    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False

    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    checkpoint = ModelCheckpoint("samplex_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=10, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=30, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])

    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


Epoch 1/30
15/15 [==============================] - ETA: 0s - loss: 1.7221 - accuracy: 0.6460
Epoch 1: val_accuracy improved from -inf to 0.63158, saving model to samplex_efficientnet.h5
15/15 [==============================] - 16s 674ms/step - loss: 1.7221 - accuracy: 0.6460 - val_loss: 1.4595 - val_accuracy: 0.6316
Epoch 2/30
15/15 [==============================] - ETA: 0s - loss: 1.3420 - accuracy: 0.7257
Epoch 2: val_accuracy improved from 0.63158 to 0.64912, saving model to samplex_efficientnet.h5
15/15 [==============================] - 10s 669ms/step - loss: 1.3420 - accuracy: 0.7257 - val_loss: 2.1782 - val_accuracy: 0.6491
Epoch 3/30
15/15 [==============================] - ETA: 0s - loss: 0.0956 - accuracy: 0.9469
Epoch 3: val_accuracy did not improve from 0.64912
15/15 [==============================] - 10s 654ms/step - loss: 0.0956 - accuracy: 0.9469 - val_loss: 1.9684 - val_accuracy: 0.6316
Epoch 4/30
15/15 [==============================] - ETA: 0s - loss: 0.1692 - accur

In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 133
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]

    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False

    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    checkpoint = ModelCheckpoint("samplex_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=10, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=30, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])

    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


Epoch 1/30
17/17 [==============================] - ETA: 0s - loss: 3.5818 - accuracy: 0.5113
Epoch 1: val_accuracy improved from -inf to 0.66667, saving model to samplex_efficientnet.h5
17/17 [==============================] - 17s 676ms/step - loss: 3.5818 - accuracy: 0.5113 - val_loss: 1.7110 - val_accuracy: 0.6667
Epoch 2/30
17/17 [==============================] - ETA: 0s - loss: 0.5434 - accuracy: 0.8195
Epoch 2: val_accuracy did not improve from 0.66667
17/17 [==============================] - 10s 619ms/step - loss: 0.5434 - accuracy: 0.8195 - val_loss: 2.3690 - val_accuracy: 0.6491
Epoch 3/30
17/17 [==============================] - ETA: 0s - loss: 0.2392 - accuracy: 0.9474
Epoch 3: val_accuracy improved from 0.66667 to 0.73684, saving model to samplex_efficientnet.h5
17/17 [==============================] - 10s 631ms/step - loss: 0.2392 - accuracy: 0.9474 - val_loss: 1.3521 - val_accuracy: 0.7368
Epoch 4/30
17/17 [==============================] - ETA: 0s - loss: 0.0934 - accur

In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 84
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]

    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False

    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    checkpoint = ModelCheckpoint("samplex_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=10, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=30, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])

    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


Epoch 1/30
11/11 [==============================] - ETA: 0s - loss: 2.0880 - accuracy: 0.5595
Epoch 1: val_accuracy improved from -inf to 0.57895, saving model to samplex_efficientnet.h5
11/11 [==============================] - 15s 935ms/step - loss: 2.0880 - accuracy: 0.5595 - val_loss: 1.6668 - val_accuracy: 0.5789
Epoch 2/30
11/11 [==============================] - ETA: 0s - loss: 0.5254 - accuracy: 0.8929
Epoch 2: val_accuracy improved from 0.57895 to 0.59649, saving model to samplex_efficientnet.h5
11/11 [==============================] - 7s 671ms/step - loss: 0.5254 - accuracy: 0.8929 - val_loss: 3.8920 - val_accuracy: 0.5965
Epoch 3/30
11/11 [==============================] - ETA: 0s - loss: 0.4006 - accuracy: 0.9167
Epoch 3: val_accuracy improved from 0.59649 to 0.64912, saving model to samplex_efficientnet.h5
11/11 [==============================] - 8s 766ms/step - loss: 0.4006 - accuracy: 0.9167 - val_loss: 2.0892 - val_accuracy: 0.6491
Epoch 4/30
11/11 [=====================

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-c130aead0ee0>", line 42, in <cell line: 17>
    y_pred = model.predict(x_testall)
  File "/usr/local/lib/python3.9/dist-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras/engine/training.py", line 2033, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/usr/local/lib/python3.9/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/tensorflow/python/eager/def_function.py", line 915, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.9/dist-packages/tensorflow/python/eager/def_function.py", line 986, in _call
  

TypeError: ignored

In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 74
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]

    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False

    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    checkpoint = ModelCheckpoint("samplex_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=10, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=30, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])

    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 101
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]

    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False

    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    checkpoint = ModelCheckpoint("samplex_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=10, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=30, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])

    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 69
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]

    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False

    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    checkpoint = ModelCheckpoint("samplex_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=10, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=30, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])

    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)
